In [20]:
import numpy as np
import pandas as pd
import yaml

In [21]:
def open_result(dataset, experiment, moo):
    with open('_'.join([dataset, experiment, moo])+'.yaml', 'r') as f:
        doc = yaml.load(f, Loader=yaml.SafeLoader)
        #data = yaml.dump(doc, sort_keys=True)
    return doc

In [22]:
datasets = ['german', 'giveme', 'taiwan', 'pima', 'student']

## Table 1

In [23]:
table_1 = pd.DataFrame(columns=datasets, index=['mi-logistic', 'greedy', 'MAPOCAM'])

In [24]:
for dataset in table_1.columns:
    data_ = open_result(dataset, 'logistic', 'MPC cost')
    for method in table_1.index:
        #print(table_1.loc[method, dataset], data_['Time performance'][method])
        table_1.loc[method, dataset] = data_['Time performance'][method]

In [25]:
table_1.astype(float).round({'giveme': 3, 'german':2, 'taiwan':2,'pima':2,'student':2})

,german,giveme,taiwan,pima,student
mi-logistic,0.02,0.015,0.02,0.02,0.03
greedy,0.01,0.020,0.03,0.02,0.03
MAPOCAM,0.02,0.004,0.05,0.01,0.04


In [26]:
print(table_1.astype(float).round({'giveme': 3, 'german':2, 'taiwan':2,'pima':2,'student':2}).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  german &  giveme &  taiwan &  pima &  student \\
\midrule
mi-logistic &    0.02 &   0.015 &    0.02 &  0.02 &     0.03 \\
greedy      &    0.01 &   0.020 &    0.03 &  0.02 &     0.03 \\
MAPOCAM     &    0.02 &   0.004 &    0.05 &  0.01 &     0.04 \\
\bottomrule
\end{tabular}



## Table 2

In [27]:
arrays = [[moo for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-logistic', 'brute-force', 'MAPOCAM']],
          [method for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-logistic', 'brute-force', 'MAPOCAM']]
         ]
index = pd.MultiIndex.from_arrays(arrays, names=('conflict', 'method'))
table_2 = pd.DataFrame(columns=datasets, index=index)

In [28]:
for dataset in table_2.columns:
    for conflict, method in table_2.index:
        try:
            data_ = open_result(dataset, 'logistic', conflict)
            table_2.loc[(conflict, method), dataset] = data_['Time performance'][method]
        except:
            pass

In [29]:
table_2.astype(float).round(2)

german  giveme  taiwan  pima  student
conflict         method                                               
MPC vs. #changes po-mi-logistic    0.01    0.01    0.01  0.01     0.02
                 brute-force       0.88    2.22    1.79  0.81     1.46
                 MAPOCAM           0.12    0.01    0.13  0.02     0.13
feature          po-mi-logistic    0.54    0.05    0.55  0.26     0.83
                 brute-force       0.60    1.41    1.19  0.71     1.26
                 MAPOCAM           0.21    0.01    0.16  0.05     0.27

In [30]:
print(table_2.astype(float).round(2).to_latex())

\begin{tabular}{llrrrrr}
\toprule
        &         &  german &  giveme &  taiwan &  pima &  student \\
conflict & method &         &         &         &       &          \\
\midrule
MPC vs. \#changes & po-mi-logistic &    0.01 &    0.01 &    0.01 &  0.01 &     0.02 \\
        & brute-force &    0.88 &    2.22 &    1.79 &  0.81 &     1.46 \\
        & MAPOCAM &    0.12 &    0.01 &    0.13 &  0.02 &     0.13 \\
feature & po-mi-logistic &    0.54 &    0.05 &    0.55 &  0.26 &     0.83 \\
        & brute-force &    0.60 &    1.41 &    1.19 &  0.71 &     1.26 \\
        & MAPOCAM &    0.21 &    0.01 &    0.16 &  0.05 &     0.27 \\
\bottomrule
\end{tabular}



## Table 3

In [31]:
arrays = [[moo for moo in ['MPC cost']
           for method in ['mi-trees', 'greedy', 'MAPOCAM-N', 'MAPOCAM-P']]+
          [moo for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-trees', 'brute-force', 'MAPOCAM-N', 'MAPOCAM-P']],
          [method for moo in ['MPC cost']
           for method in ['mi-trees', 'greedy', 'MAPOCAM-N', 'MAPOCAM-P']]+
          [method for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-trees', 'brute-force', 'MAPOCAM-N', 'MAPOCAM-P']]
         ]
index = pd.MultiIndex.from_arrays(arrays, names=('conflict', 'method'))
table_3 = pd.DataFrame(columns=datasets, index=index)

In [32]:
for dataset in table_3.columns:
    for conflict, method in table_3.index:
        try:
            data_ = open_result(dataset, 'tree', conflict)
            table_3.loc[(conflict, method), dataset] = data_['Time performance'][method]
        except:
            pass

In [33]:
table_3.astype(float).round(2)

german  giveme  taiwan  pima  student
conflict         method                                            
MPC cost         mi-trees       0.02    0.02    0.02  0.02     0.02
                 greedy         0.01    0.01    0.02  0.02     0.01
                 MAPOCAM-N      0.10    0.01   22.47  1.56     0.06
                 MAPOCAM-P      0.04    0.01    0.03  0.17     0.01
MPC vs. #changes po-mi-trees    0.06    0.02    0.08  0.12     0.02
                 brute-force    1.25    0.34    1.25  1.42     0.87
                 MAPOCAM-N      0.38    0.01    0.75  0.28     0.06
                 MAPOCAM-P      0.18    0.01    0.04  0.08     0.03
feature          po-mi-trees    0.05    0.03    0.09  0.12     0.02
                 brute-force    0.47    0.21    0.81  1.01     0.19
                 MAPOCAM-N      0.26    0.06    0.65  0.49     0.06
                 MAPOCAM-P      0.18    0.01    0.05  0.16     0.04

In [34]:
print(table_3.astype(float).round(2).to_latex())

\begin{tabular}{llrrrrr}
\toprule
        &           &  german &  giveme &  taiwan &  pima &  student \\
conflict & method &         &         &         &       &          \\
\midrule
MPC cost & mi-trees &    0.02 &    0.02 &    0.02 &  0.02 &     0.02 \\
        & greedy &    0.01 &    0.01 &    0.02 &  0.02 &     0.01 \\
        & MAPOCAM-N &    0.10 &    0.01 &   22.47 &  1.56 &     0.06 \\
        & MAPOCAM-P &    0.04 &    0.01 &    0.03 &  0.17 &     0.01 \\
MPC vs. \#changes & po-mi-trees &    0.06 &    0.02 &    0.08 &  0.12 &     0.02 \\
        & brute-force &    1.25 &    0.34 &    1.25 &  1.42 &     0.87 \\
        & MAPOCAM-N &    0.38 &    0.01 &    0.75 &  0.28 &     0.06 \\
        & MAPOCAM-P &    0.18 &    0.01 &    0.04 &  0.08 &     0.03 \\
feature & po-mi-trees &    0.05 &    0.03 &    0.09 &  0.12 &     0.02 \\
        & brute-force &    0.47 &    0.21 &    0.81 &  1.01 &     0.19 \\
        & MAPOCAM-N &    0.26 &    0.06 &    0.65 &  0.49 &     0.06 \\
        

## Table 4

In [35]:
arrays = [[moo for moo in ['MPC cost']
           for method in ['mi-trees', 'greedy', 'MAPOCAM-N', 'MAPOCAM-P', 'MAPOCAM-M']]+
          [moo for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-trees', 'brute-force', 'MAPOCAM-N', 'MAPOCAM-P', 'MAPOCAM-M']],
          [method for moo in ['MPC cost']
           for method in ['mi-trees', 'greedy', 'MAPOCAM-N', 'MAPOCAM-P', 'MAPOCAM-M']]+
          [method for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-trees', 'brute-force', 'MAPOCAM-N', 'MAPOCAM-P', 'MAPOCAM-M']]
         ]
index = pd.MultiIndex.from_arrays(arrays, names=('conflict', 'method'))
table_4 = pd.DataFrame(columns=datasets, index=index)

In [36]:
for dataset in table_4.columns:
    for conflict, method in table_4.index:
        try:
            data_ = open_result(dataset, 'monotone-tree', conflict)
            table_4.loc[(conflict, method), dataset] = data_['Time performance'][method]
        except:
            pass

In [37]:
table_4.astype(float).round(2)

german  giveme  taiwan  pima  student
conflict         method                                            
MPC cost         mi-trees       0.02    0.03    0.06  0.05     0.02
                 greedy         0.02    0.02    0.07  0.03     0.01
                 MAPOCAM-N      1.32    0.01    2.77  0.58     0.05
                 MAPOCAM-P      2.68    0.02    6.86  1.04     0.10
                 MAPOCAM-M      0.03    0.01    0.05  0.02     0.01
MPC vs. #changes po-mi-trees    0.13    0.04    0.76  0.06     0.02
                 brute-force    1.43    0.67   16.83  0.81     0.61
                 MAPOCAM-N      0.41    0.01    0.69  0.22     0.08
                 MAPOCAM-P      0.56    0.02    1.31  0.32     0.11
                 MAPOCAM-M      0.08    0.01    0.04  0.02     0.01
feature          po-mi-trees    0.13    0.04    0.76  0.06     0.03
                 brute-force    0.61    0.46   14.05  0.58     0.29
                 MAPOCAM-N      0.32    0.07   11.50  0.31     0.21
                 MAPOCAM-P      0.79    0.16   19.95  0.49     0.44
                 MAPOCAM-M      0.09    0.01    0.12  0.02     0.02

In [38]:
print(table_4.astype(float).round(2).to_latex())

\begin{tabular}{llrrrrr}
\toprule
        &           &  german &  giveme &  taiwan &  pima &  student \\
conflict & method &         &         &         &       &          \\
\midrule
MPC cost & mi-trees &    0.02 &    0.03 &    0.06 &  0.05 &     0.02 \\
        & greedy &    0.02 &    0.02 &    0.07 &  0.03 &     0.01 \\
        & MAPOCAM-N &    1.32 &    0.01 &    2.77 &  0.58 &     0.05 \\
        & MAPOCAM-P &    2.68 &    0.02 &    6.86 &  1.04 &     0.10 \\
        & MAPOCAM-M &    0.03 &    0.01 &    0.05 &  0.02 &     0.01 \\
MPC vs. \#changes & po-mi-trees &    0.13 &    0.04 &    0.76 &  0.06 &     0.02 \\
        & brute-force &    1.43 &    0.67 &   16.83 &  0.81 &     0.61 \\
        & MAPOCAM-N &    0.41 &    0.01 &    0.69 &  0.22 &     0.08 \\
        & MAPOCAM-P &    0.56 &    0.02 &    1.31 &  0.32 &     0.11 \\
        & MAPOCAM-M &    0.08 &    0.01 &    0.04 &  0.02 &     0.01 \\
feature & po-mi-trees &    0.13 &    0.04 &    0.76 &  0.06 &     0.03 \\
        & 